In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/otto_train.csv") # Product Category
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
'''
id: 고유 아이디
feat_1 ~ feat_93: 설명변수
target: 타겟변수 (1~9)
'''

'\nid: 고유 아이디\nfeat_1 ~ feat_93: 설명변수\ntarget: 타겟변수 (1~9)\n'

In [5]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 95


## 의미가 없다고 판단되는 변수 제거

In [6]:
data = data.drop(['id'], axis = 1) # id 제거

## 타겟 변수의 문자열을 숫자로 변환

In [7]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [9]:
# !pip install xgboost
import xgboost as xgb
import time
start = time.time() # 시작 시간 지정
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환
xgb_param = {'max_depth': 10, # 트리 깊이
         'learning_rate': 0.01, # Step Size
         'n_estimators': 100, # Number of trees, 트리 생성 개수
         'objective': 'multi:softmax', # 목적 함수
        'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

[14:21:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 76.67 %
Time: 36.74 seconds


In [11]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

def train_model(train_x, train_y, test_x, test_y):
    xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
    xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환
    xgb_param = {'max_depth': 10, # 트리 깊이
             'learning_rate': 0.01, # Step Size
             'n_estimators': 100, # Number of trees, 트리 생성 개수
             'objective': 'multi:softmax', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
    xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    pred_mae = mean_absolute_error(test_y, xgb_model_predict)
    
    return pred_mae

In [12]:
train_model(train_x, train_y, test_x, test_y)

[15:52:27] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.5762766645119586

In [10]:
xgb_model_predict

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [11]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclass', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 73.57 %
Time: 18.88 seconds


In [12]:
lgb_model.predict(test_x)

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [13]:
# !pip install catboost
import catboost as cb
start = time.time() # 시작 시간 지정
cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

0:	learn: 0.5907034	total: 1.3s	remaining: 2m 8s
1:	learn: 0.6356107	total: 2.61s	remaining: 2m 8s
2:	learn: 0.6411256	total: 3.87s	remaining: 2m 5s
3:	learn: 0.6480344	total: 4.99s	remaining: 1m 59s
4:	learn: 0.6508222	total: 6.43s	remaining: 2m 2s
5:	learn: 0.6499939	total: 8.03s	remaining: 2m 5s
6:	learn: 0.6507818	total: 10s	remaining: 2m 13s
7:	learn: 0.6548422	total: 11.8s	remaining: 2m 15s
8:	learn: 0.6559533	total: 13.7s	remaining: 2m 18s
9:	learn: 0.6560947	total: 15.5s	remaining: 2m 19s
10:	learn: 0.6568421	total: 17.3s	remaining: 2m 19s
11:	learn: 0.6588219	total: 19s	remaining: 2m 19s
12:	learn: 0.6592259	total: 20.8s	remaining: 2m 19s
13:	learn: 0.6611248	total: 22.7s	remaining: 2m 19s
14:	learn: 0.6625591	total: 24.5s	remaining: 2m 18s
15:	learn: 0.6631853	total: 26.3s	remaining: 2m 18s
16:	learn: 0.6639328	total: 28.2s	remaining: 2m 17s
17:	learn: 0.6668821	total: 30s	remaining: 2m 16s
18:	learn: 0.6669630	total: 31.6s	remaining: 2m 14s
19:	learn: 0.6675286	total: 33.6s	

In [14]:
cb_model.predict(test_x)

array([[-0.35426047,  1.22109587,  0.44230101, ..., -0.1698448 ,
        -0.02059177, -0.2130643 ],
       [-0.07235138,  0.42535181,  0.20060428, ...,  0.21863604,
         0.2719157 ,  0.25089315],
       [-0.3315885 , -0.31862353, -0.31279765, ..., -0.29798357,
        -0.24018767, -0.32984969],
       ...,
       [ 0.05304325,  0.02500267, -0.14752573, ..., -0.20741963,
         0.12789417,  1.51166757],
       [-0.55093666,  1.7691278 ,  0.99746884, ..., -0.3420542 ,
        -0.49799871, -0.38136323],
       [-0.3033724 ,  0.09352675, -0.11808658, ...,  0.65825036,
         1.05515787, -0.20799899]])

- 자기가 하는 데이터분석의 목적이 regression(회귀)이냐 classification(분류)아냐, classification 중에서도 multi class냐 binary냐 이거에 따라서 바뀌는 파라미터가 **'objective'나 'eval_metric'**. 이 부분만 바뀌고 나머지 하이퍼 파라미터(max_depth, learning_rate 등등)은 똑같이 사용할 수 있음
<br>
<br>
- 일반적인 성능 : LightGBM > CatBoost > XGBoost

In [15]:
# 집 값(price) 예측을 LightGBM으로 해보자

# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [16]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

In [17]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [18]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행


In [19]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))
# 단일모델(LightGBM)로 했을 때 mse

210904.17249451784

## Ensemble의 Ensemble

In [20]:
# Ensemble의 Ensemble은 bagging 기법 사용 (복원추출해서 fitting하는 것 여러번 반복)

import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30): 
    # for문 이유 : 샘플링을 다르게 한다거나 변수를 좀 다르게 하려고
    # 30번 for문 돌 때마다 데이터 복원추출하고 그 데이터 통해 lgb용 데이터 만들고 모델링시키고 예측
    
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index], label = train_y.iloc[random_data_index]) # 학습 데이터를 LightGBM 모델에 맞게 변환
    lgb_param = {'max_depth': 10, # 트리 깊이
                'learning_rate': 0.01, # Step Size
                'n_estimators': 500, # Number of trees, 트리 생성 개수
                'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    # for문 돌 때마다 하이퍼 파라미터를 다르게 설정해볼까?
    # 데이터를 복원추출했는데 변수도 복원추출하면 더 좋아지지않을까?
    # 이런 생각으로 실제로 경험을 해보면 실력을 쌓을 수 있겠지
    
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
    # 여기에 lightgbm뿐만 아니라 xgboost, catboost넣을 수도 있고
    # 또다른 SVM, KNN, Decision Tree, Lasso, Ridge 등도 같이 섞어서 앙상블 해볼 수 있겠지
    # 다양함이 앙상블의 기본적인 조건이니까!
    
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
    print(sqrt(mean_squared_error(lgb_model.predict(test_x),test_y)))

9522


C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


211758.6502638526
9577


C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


213806.13847698047
9505


...

212988.31672198008
9533


C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


211241.98986839628
9568


C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


216272.88168991246


In [21]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [22]:
bagging_predict

[504218.7820307572,
 632681.0869227976,
 950145.1354093045,
 1594505.4687036262,
 637638.5702198489,
 368742.51175494667,
 703524.1813378875,
 430307.44734305417,
 463311.78064029774,
 494963.87497308565,
 635022.2027665002,
 381849.71009199444,
 299795.9230156198,
 359537.98415135685,
 342486.76715633646,
 1298870.6130628025,
 371654.6087104884,
 1048323.5026587471,
 317675.83934992267,
 527702.524051179,
 379591.6830357145,
 1939808.497802569,
 665632.7681158979,
 534426.9573840115,
 504027.29933619,
 485463.2460633716,
 297038.4246375748,
 255677.09097264113,
 477675.7980128618,
 534210.845562278,
 494784.7999451674,
 471547.4072311538,
 467397.43205417116,
 585414.0444801948,
 378181.18568272854,
 1036239.8374627909,
 899481.6954295987,
 533543.208424743,
 357600.11279569636,
 1572378.8058978287,
 394009.1495373982,
 277440.54551530903,
 512535.0068452253,
 341378.83548352367,
 252380.92818879525,
 244647.4167346164,
 330397.010165253,
 330959.2888530904,
 355019.9894553494,
 56991

In [23]:
sqrt(mean_squared_error(bagging_predict,test_y))
# 위에 lbm 단일모델로 한 것보다 성능이 더 좋아졌지
# 복원추출은 할 때마다 결과가 바뀌기때문에 여러 번 확인해봐야함
# 강사님은 Ensemble의 Ensemble 기법을 자주 쓴대
# 기본적으로 먼저 lgb이나 catboost, xgboost 해놓고
# 전체데이터를 다 넣고 모델 fitting해서 성능보고 Ensemble의 Ensembled하면
# 8~90%는 성능이 좋아짐

210075.22678067334